In [1]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize
import math

In [2]:
text = '''If you like tuna and tomato sauce- try combining the two.
         It's really not as bad as it sounds.
         If the Easter Bunny and the Tooth Fairy had babies would they take your teeth and leave chocolate for you?'''

In [3]:
sentences = sent_tokenize(text)

print(sentences)

['If you like tuna and tomato sauce- try combining the two.', "It's really not as bad as it sounds.", 'If the Easter Bunny and the Tooth Fairy had babies would they take your teeth and leave chocolate for you?']


In [4]:
def remove_string_special_characters(s):
    
    # replace special characters with
    stripped = re.sub('[^\w\s]', '',s)
    stripped = re.sub('_','',stripped)
    
    # Change any whitespace to one space
    stripped = re.sub('\s+',' ',stripped)
    
    # Remove start and white space
    stripped = stripped.strip()
    
    return stripped
                      

In [5]:
sentences = [remove_string_special_characters(s) for s in sentences]

sentences

['If you like tuna and tomato sauce try combining the two',
 'Its really not as bad as it sounds',
 'If the Easter Bunny and the Tooth Fairy had babies would they take your teeth and leave chocolate for you']

In [6]:
doc_info = []

i =0

for sent in sentences:
    i = i+1
    count  = len(word_tokenize(sent))
    temp = {'dpc_id': i ,'doc_length': count}
    doc_info.append(temp)

In [7]:
doc_info

[{'dpc_id': 1, 'doc_length': 11},
 {'dpc_id': 2, 'doc_length': 8},
 {'dpc_id': 3, 'doc_length': 20}]

In [8]:
i =0;
freqDict_list = []
for sent in sentences:
    i = i+1
    freq_dict = {}
    words = word_tokenize(sent)
    for word in words:
        word = word.lower()
        if word in freq_dict:
            freq_dict[word] +=1
        else:
            freq_dict[word] = 1
        temp = {'doc_id':i,'freq_dict':freq_dict}
    freqDict_list.append(temp)

In [9]:
# Compute TF: (Total number of times the word occurs in the Text / Total number of words in text)
Tf_scores = []

for tempDict in freqDict_list:
    id = tempDict['doc_id']
    for k in tempDict['freq_dict']:
        temp = {'doc_id':id,
                'Tf_score':tempDict['freq_dict'][k] / doc_info[id-1]['doc_length'],
                'key':k}
        Tf_scores.append(temp)

In [10]:
# Compute IDF : (Total number of documents / Number of documensts with word t in it)

TDF_scores = []
counter =0
for dict in freqDict_list:
    counter += 1
    

In [11]:
import pandas as pd

df = pd.DataFrame(sentences,columns=['sentences'])

df['sentences_wordTokenize'] = df['sentences'].apply(lambda x : word_tokenize(x) )

In [12]:
df['word_lenth'] = df['sentences_wordTokenize'].apply(lambda x : len(x))

In [13]:
def getFreq_Dict(sent):
    freq_dict = {}
    for w in sent:
        if w in freq_dict:
            freq_dict[w] +=1
        else:
            freq_dict[w] = 1
    return freq_dict

In [14]:
df['freq_dict'] = df['sentences_wordTokenize'].apply(lambda x : getFreq_Dict(x))

In [15]:
def getTF_Sore(freq_dict):
    tf_scores = {}
    
    total_numbr_words = sum(freq_dict.values())
    
    words = list(freq_dict.keys())
    
    for i in range(len(words)):
        tf = freq_dict[words[i]] / total_numbr_words
        tf_scores[words[i]] = tf
        
    return tf_scores

In [16]:
df['Tf_score'] = df['freq_dict'].apply(lambda x : getTF_Sore(x))

In [17]:
def computeIDF(doc_info,freqDict_list):
    IDF_scores = {}
    counter = 0
    for dict in freqDict_list:
        counter +=1
        for k in dict['freq_dict'].keys():
            count = sum([k in tempDict['freq_dict'] for tempDict in freqDict_list])
            
            IDF_scores[k] = math.log(len(doc_info)/count)
            
    return IDF_scores


In [18]:
idf = computeIDF(doc_info,freqDict_list)

In [19]:
def IDFCal(sent_words):
    idf_scores = {}
    words = list(sent_words.keys())
    for i in range(len(words)):
        idf_scores[words[i]] = idf[words[i].lower()]
        
    return idf_scores

In [20]:
df['IDF_score'] = df['Tf_score'].apply(lambda x : IDFCal(x))

In [22]:
idf_score = {}

for dict in list(df['IDF_score']):
    words = list(dict.keys())
    for i in range(len(words)):
        idf_score[words[i]] = dict[words[i]]

In [25]:
def computeTfIDF(tf_score):
    words = list(tf_score.keys())
    
    tf_idf_score = {}
    
    for i in range(len(words)):
        tf_sre = tf_score[words[i]]
        idf_sre = idf_score[words[i]]
        
        tf_idf_score[words[i]] = tf_sre * idf_sre
        
    return tf_idf_score

In [26]:
df['TF_IDF'] = df['Tf_score'].apply(lambda x : computeTfIDF(x))

In [27]:
df

,sentences,sentences_wordTokenize,word_lenth,freq_dict,Tf_score,IDF_score,TF_IDF
0,If you like tuna and tomato sauce try combinin...,"[If, you, like, tuna, and, tomato, sauce, try,...",11,"{'If': 1, 'you': 1, 'like': 1, 'tuna': 1, 'and...","{'If': 0.09090909090909091, 'you': 0.090909090...","{'If': 0.4054651081081644, 'you': 0.4054651081...","{'If': 0.036860464373469494, 'you': 0.03686046..."
1,Its really not as bad as it sounds,"[Its, really, not, as, bad, as, it, sounds]",8,"{'Its': 1, 'really': 1, 'not': 1, 'as': 2, 'ba...","{'Its': 0.125, 'really': 0.125, 'not': 0.125, ...","{'Its': 1.0986122886681098, 'really': 1.098612...","{'Its': 0.13732653608351372, 'really': 0.13732..."
2,If the Easter Bunny and the Tooth Fairy had ba...,"[If, the, Easter, Bunny, and, the, Tooth, Fair...",20,"{'If': 1, 'the': 2, 'Easter': 1, 'Bunny': 1, '...","{'If': 0.05, 'the': 0.1, 'Easter': 0.05, 'Bunn...","{'If': 0.4054651081081644, 'the': 0.4054651081...","{'If': 0.02027325540540822, 'the': 0.040546510..."
